# Encoder-Decoderによる機械翻訳

---
## 目的
Encoder-Decoder構造のネットワークである（Sequence-to-Sequenel Seq2Seq）を用いて機械翻訳（English-French）を行う．


## 対応するチャプター
* 10.4: Encoder-DecoderとSequence-to-Sequence

## データのダウンロード
実習に必要なデータをダウンロードします．
下記のコードを実行してデータのダウンロードを行ってください．

In [ ]:
!wget http://www.mprg.cs.chubu.ac.jp/~hirakawa/share/tutorial_data/fra-eng-tr_data.zip
!unzip -q -o fra-eng-tr_data.zip
!ls ./fra-eng-tr_data

## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．

In [ ]:
from time import time
import json
import numpy
import numpy as np

import chainer
from chainer.datasets import get_ptb_words, get_ptb_words_vocabulary
from chainer import cuda
from chainer import Variable
import chainer.functions as F
import chainer.links as L
from chainer.optimizer_hooks import GradientClipping

## データセットの読み込み
データセットを読み込みます．
今回のデータはフランス語と英語の対訳データセットを使用します．
使用するデータには，学習データとして8479，テストデータとして2120の対訳文が含まれています．

また，このデータセットに存在する単語の情報を取得します．
`fra_vocab`にはフランス語の単語情報，`eng_vocab`には英語の単語情報を含んだ辞書オブジェクトが読み込まれます．

In [ ]:
def download_text_dataset(input_filename):
    downloaded = []
    with open(input_filename) as f:
        for s in f.readlines():
            downloaded.append( np.array(list(map(int, s.strip().split(' '))), dtype=np.int32) )
    return downloaded

fra_train = download_text_dataset("fra-eng-tr_data/fra_train.txt")
eng_train = download_text_dataset("fra-eng-tr_data/eng_train.txt")
fra_test = download_text_dataset("fra-eng-tr_data/fra_test.txt")
eng_test = download_text_dataset("fra-eng-tr_data/eng_test.txt")

print(len(fra_train))
print(len(fra_test))
        
with open("fra-eng-tr_data/fra_vocab.json") as f:
    fra_vocab = json.load(f)
    fra_vocab = {int(k):v for k, v in fra_vocab.items()}
    
with open("fra-eng-tr_data/eng_vocab.json") as f:
    eng_vocab = json.load(f)
    eng_vocab = {int(k):v for k, v in eng_vocab.items()}

## ネットワークモデルの定義
Seq2Seqのネットワークを定義します．

文章の終わりを表現する単語IDとして`EOS=1`を定義します．

次に，`sequence_embed`関数を定義します．
この関数では，この次に定義するネットワーク内部での計算を行う際に，任意の長さの文章をまとめたミニバッチデータに対して，ある処理を行うための関数です．
この関数を用いてネットワークの演算を行うことで，ミニバッチ中に含まれる文章の長さが異なる場合でも，一度に処理を行うことが可能となります．

翻訳を行うための`Seq2seq`ネットワークを定義します．
このネットワークには，入力された文章（フランス語文）を処理するEncoderと翻訳結果（英文）を出力するDecoderが存在します．

まず`__init__`関数でネットワーク内部の層を定義します．
`__init__`関数の引数として，次の引数を準備します．
`n_layers`はEncoderおよびDecoderに用いるLSTMの総数，`n_source_vocab`は入力側で扱う単語数，`n_target_vocab`は出力側で扱う単語数，`n_units`はLSTMの隠れ層のユニット数です．
`embed_x`および`embed_y`ではそれぞれ，入力・出力側で扱う単語のIDから単語を表現した特徴ベクトルを出力するためのembed層を定義します．
次に，`encoder`および`decoder`では，LSTM層を定義します．
ここでは，`NStepLSTM`という層を使用して定義を行います．
`NStepLSTM`は，任意の長さのデータ（文章）をまとめたミニバッチを同時に扱うことができるLSTMの定義方法です．
ここでは，LSTMの総数やdropoutのdrop率も同時に定義することが可能です．
最後に，`decoder`から出力された特徴を元に翻訳結果の単語IDを出力するための出力層`W`を定義します．

次に，入力されたデータから結果を出力するための演算を定義します．
ここでは，下記で実行する学習のプログラムを簡単にするため，2種類の演算の関数を定義します．
`__call__`関数では，学習を行う際の演算を定義します．ネットワークの出力結果と正解の翻訳情報から計算した誤差を返す関数として定義します．
また，`translate`関数では，ネットワークが予測した翻訳結果を出力するように定義します．

In [ ]:
EOS = 1

def sequence_embed(embed, xs):
    x_len = [len(x) for x in xs]
    x_section = np.cumsum(x_len[:-1])
    ex = embed(F.concat(xs, axis=0))
    exs = F.split_axis(ex, x_section, 0)
    return exs

class Seq2seq(chainer.Chain):

    def __init__(self, n_layers, n_source_vocab, n_target_vocab, n_units):
        super(Seq2seq, self).__init__()
        with self.init_scope():
            self.embed_x = L.EmbedID(n_source_vocab, n_units)
            self.embed_y = L.EmbedID(n_target_vocab, n_units)
            self.encoder = L.NStepLSTM(n_layers, n_units, n_units, 0.1)
            self.decoder = L.NStepLSTM(n_layers, n_units, n_units, 0.1)
            self.W = L.Linear(n_units, n_target_vocab)

        self.n_layers = n_layers
        self.n_units = n_units

    def __call__(self, xs, ys):
        xs = [x[::-1] for x in xs]

        eos = self.xp.array([EOS], np.int32)
        ys_in = [F.concat([eos, y], axis=0) for y in ys]
        ys_out = [F.concat([y, eos], axis=0) for y in ys]

        # Both xs and ys_in are lists of arrays.
        exs = sequence_embed(self.embed_x, xs)
        eys = sequence_embed(self.embed_y, ys_in)

        batch = len(xs)
        # None represents a zero vector in an encoder.
        hx, cx, _ = self.encoder(None, None, exs)
        _, _, os = self.decoder(hx, cx, eys)

        # It is faster to concatenate data before calculating loss
        # because only one matrix multiplication is called.
        concat_os = F.concat(os, axis=0)
        concat_ys_out = F.concat(ys_out, axis=0)
        loss = F.sum(F.softmax_cross_entropy(
            self.W(concat_os), concat_ys_out, reduce='no')) / batch

        n_words = concat_ys_out.shape[0]
        perp = self.xp.exp(loss.array * batch / n_words)
        return loss

    def translate(self, xs, max_length=100):
        batch = len(xs)
        with chainer.no_backprop_mode(), chainer.using_config('train', False):
            xs = [x[::-1] for x in xs]
            exs = sequence_embed(self.embed_x, xs)
            h, c, _ = self.encoder(None, None, exs)
            ys = self.xp.full(batch, EOS, np.int32)
            result = []
            for i in range(max_length):
                eys = self.embed_y(ys)
                eys = F.split_axis(eys, batch, 0)
                h, c, ys = self.decoder(h, c, eys)
                cys = F.concat(ys, axis=0)
                wy = self.W(cys)
                ys = self.xp.argmax(wy.array, axis=1).astype(np.int32)
                result.append(ys)

        return result

## ネットワークの作成
上のプログラムで定義したネットワークを作成します．
ここでは，GPUで学習を行うために，modelをGPUに送るto_gpu関数を利用しています．

学習を行う際の最適化方法としてモーメンタムSGD(モーメンタム付き確率的勾配降下法）を利用します．また，学習率を0.001として引数に与えます．そして，最適化方法のsetup関数にネットワークモデルを与えます．

In [ ]:
num_source_vocab = len(fra_vocab)
num_target_vocab = len(eng_vocab)
model = Seq2seq(n_layers=2,
                n_source_vocab=num_source_vocab,
                n_target_vocab=num_target_vocab,
                n_units=1024)
model.to_gpu()

optimizer = chainer.optimizers.MomentumSGD(lr=0.001, momentum=0.9)
optimizer.setup(model)
optimizer.add_hook(GradientClipping(5.0))

## 学習

学習を実行します．

In [ ]:
xp = cuda.cupy

# ミニバッチサイズ・エポック数．学習データ数の設定
batch_size = 32
epoch_num = 100
train_data_num = len(fra_train)
num_iter_per_epoch = int(train_data_num / batch_size)

# 学習の実行
start = time()
for epoch in range(1, epoch_num + 1):
    
    sum_loss = 0
    
    perm = np.random.permutation(train_data_num)
    for i in range(0, train_data_num, batch_size):

        fra_batch = [ Variable(cuda.to_gpu(np.array(fra_train[ii], dtype=np.int32))) for ii in perm[i:i+batch_size] ]
        eng_batch = [ Variable(cuda.to_gpu(np.array(eng_train[ii], dtype=np.int32))) for ii in perm[i:i+batch_size] ]
        
        loss = model(fra_batch, eng_batch)
        
        sum_loss += loss.data

        model.cleargrads()
        loss.backward()
        optimizer.update()

    elapsed_time = time() - start
    print("epoch: {}, mean loss: {}, elapsed_time: {}".format(epoch,
                                                              sum_loss/num_iter_per_epoch,
                                                              elapsed_time))

    if epoch % 20 == 0:
        model.to_cpu()
        chainer.serializers.save_npz("model-%03d.npz" % epoch, model)
        model.to_gpu()

## テスト

学習後のネットワークを用いて，翻訳を行います．

In [ ]:
with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):
    test_index = np.arange(20)
    
    for i_test in test_index:
        fra_batch = [ Variable(cuda.to_gpu(np.array(fra_train[i_test], dtype=np.int32)))]
        eng_batch = [ np.array(eng_train[i_test], dtype=np.int32)]
    
        y = model.translate(fra_batch, max_length=10)
    
        fra_true = [fra_vocab[i] for i in cuda.to_cpu(fra_batch[0].data)]
        eng_pred = [eng_vocab[cuda.to_cpu(i)[0]] for i in y]

        print("input french sentence      :", " ".join(fra_true))
        print("translated english sentence:", " ".join(eng_pred), "\n")

## テスト（学習済みモデル）

Seq2Seqの学習には時間を要するため，下記のコードでは学習済みのモデルを読み込んで，学習後のネットワークでの翻訳結果を確認します．
保存したモデルパラメータを読み込んで，テストデータの翻訳を行います．

In [ ]:
# 学習済みモデルの読み込み
num_source_vocab = len(fra_vocab)
num_target_vocab = len(eng_vocab)
pretrained_model = Seq2seq(n_layers=2,
                n_source_vocab=num_source_vocab,
                n_target_vocab=num_target_vocab,
                n_units=1024)

chainer.serializers.load_npz("fra-eng-tr_data/seq2seq.npz", pretrained_model)
pretrained_model.to_gpu()

with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):
    test_index = np.arange(20)
    
    for i_test in test_index:
        fra_batch = [ Variable(cuda.to_gpu(np.array(fra_train[i_test], dtype=np.int32)))]
        eng_batch = [ np.array(eng_train[i_test], dtype=np.int32)]
    
        y = pretrained_model.translate(fra_batch, max_length=15)
    
        fra_true = [fra_vocab[i] for i in cuda.to_cpu(fra_batch[0].data)]
        eng_pred = [eng_vocab[cuda.to_cpu(i)[0]] for i in y]

        print("input french sentence      :", " ".join(fra_true))
        print("translated english sentence:", " ".join(eng_pred), "\n")